In [1]:
import numpy as np
import matplotlib.pyplot as plt
from make_mg_template_files import Mg_Shapes
import os
from distutils.dir_util import copy_tree
import shutil

In [2]:
main_folder = 'demo'
folders = ['hex100nm-4nm', 'kite80nm-6nm']
kind = ['hex', 'kite'] # record the shape, e.g., hex, taco, tent, kite, chair. Need new entry for each shape file.
lat_space = np.array([2, 2]) # dipole spacing 
ylength = ['100', '80'] # tip-to-tip length in nm
cluster = 'esc' # 'esc' or 'hpc'
shell = True # True = a shell, False = no shell
CRSid = 'caw97'

## Remake Shape Files from CC 

In [3]:
%rm -r ../create_files/demo/spectra/*

In [4]:
for idx, name in enumerate(folders):
    print(name)
    dirname = str('../create_files/')+str(main_folder)+str('/spectra/') + str(name)
    # Make directory, copy files
    os.mkdir(dirname)
    if shell == True: 
        copy_tree(str("template_files/")+str(cluster)+str("/"), str(dirname))
    if shell == False:
        copy_tree(str("template_files/")+str(cluster)+str("/"), str(dirname))

    # Remake shape files
    mgshape = Mg_Shapes(CRSid=CRSid,
                    lat_space=lat_space[idx], 
                    length=ylength[idx],
                    kind=kind[idx],
                    new_dir_spec=dirname,
                    new_dir_temp=str(''),
                    shell=shell)
    mgshape.rewrite_shapefile(shape_old=np.loadtxt(
        str('../create_files/')+str(main_folder)+str('/shapes_from_cc/shape-')+str(name)+str('.dat'),skiprows=7),)
    
    # Edit ddscat.par files
    mgshape.write_ddscatpar(which_cluster=cluster)
    os.rename('ddscat.par', str(dirname)+str('/ddscat.par'))

    if cluster == 'hpc':
        # Remake launch_temp
        new_launch = open(str(dirname)+str('/launch_temp.slurm'))
        lines = new_launch.readlines()
        lines[2] = str('#SBATCH -J ')+str(name) + '\n'
        new_launch = open(str(dirname)+str('/launch_temp.slurm'),"w")
        new_launch.writelines(lines)
        new_launch.close()
    
    if cluster == 'esc':
        # Remake launch_temp
        new_launch = open(str(dirname)+str('/launch_temp.slurm'))
        lines = new_launch.readlines()
        lines[1] = str('#SBATCH -J ')+str(name) + '\n'
        lines[17] = str('/home/')+str(CRSid)+str('/source_code/g-dda/source_code/ddscat &> ${file}.out &')+'\n'
        new_launch = open(str(dirname)+str('/launch_temp.slurm'),"w")
        new_launch.writelines(lines)
        new_launch.close()   
        
        # Remake submit_together
        new_submit = open(str(dirname)+str('/submit_together.slurm'))
        lines = new_submit.readlines()
        lines[17] = str('sbatch --dependency=singleton --job-name=')+str(name)+str(' collect.slurm') + '\n'
        new_submit = open(str(dirname)+str('/submit_together.slurm'),"w")
        new_submit.writelines(lines)
        new_submit.close()   
        
    # Remake make_dirs.py
    new = open(str(dirname)+str('/make_dirs.py'))
    lines = new.readlines()
    lines[11] = str('shell = ') + str(shell) +'\n'
    new = open(str(dirname)+str('/make_dirs.py'),"w")
    new.writelines(lines)
    new.close()   
    
    
    
    


hex100nm-4nm
kite80nm-6nm
